In [52]:
import pandas as pd
import os
import numpy as np
import sys
sys.path.append('..')

In [54]:
#os.chdir(r'C:\Users\Administrator\Desktop\raphi_other\repositories\template_project\src')

In [56]:
data = pd.read_csv(r'../data/raw/data/houses_data.csv')

In [58]:
#from eda_first import reduce_mem_usage
#data = reduce_mem_usage(data)

In [60]:
data.columns

Index(['bedrooms', 'bathrooms', 'toilets', 'parking_space', 'title', 'town',
       'state', 'price'],
      dtype='object')

In [62]:
data.shape

(24326, 8)

In [64]:
data.head()

,bedrooms,bathrooms,toilets,parking_space,title,town,state,price
0,6.0000,5.0000,5.0000,4.0000,Detached Duplex,Mabushi,Abuja,450000000.0000
1,4.0000,5.0000,5.0000,4.0000,Terraced Duplexes,Katampe,Abuja,800000000.0000
2,4.0000,5.0000,5.0000,4.0000,Detached Duplex,Lekki,Lagos,120000000.0000
3,4.0000,4.0000,5.0000,6.0000,Detached Duplex,Ajah,Lagos,40000000.0000
4,4.0000,4.0000,5.0000,2.0000,Semi Detached Duplex,Lekki,Lagos,75000000.0000


In [66]:
pd.set_option('display.float_format', '{:.4f}'.format)

In [68]:
data.describe()

,bedrooms,bathrooms,toilets,parking_space,price
count,24326.0000,24326.0000,24326.0000,24326.0000,24326.0000
mean,4.3388,4.6008,5.1764,4.0417,301380208.4724
std,1.1385,1.1632,1.2263,1.3999,12204027269.3809
min,1.0000,1.0000,1.0000,1.0000,90000.0000
25%,4.0000,4.0000,5.0000,4.0000,52000000.0000
50%,4.0000,5.0000,5.0000,4.0000,85000000.0000
75%,5.0000,5.0000,6.0000,4.0000,160000000.0000
max,9.0000,9.0000,9.0000,9.0000,1800000000000.0000


In [70]:
pct = np.percentile(data.loc[:, 'price'].fillna(np.mean(data.loc[:, 'price'])), 95)
print(pct)
print(data.shape)
data = data.loc[data.loc[:, 'price'] < pct]
print(data.shape)

600000000.0
(24326, 8)
(23088, 8)


In [72]:
data.columns.isnull()

array([False, False, False, False, False, False, False, False])

In [74]:
from src.eda_first import summarize_dataframe
summarize_dataframe(data)

,Column,Data Type,Unique Values,Missing Values,Sample Unique Values
0,bedrooms,float64,9,0,"[6.0, 4.0, 5.0, 2.0, 1.0]"
1,bathrooms,float64,9,0,"[5.0, 4.0, 2.0, 1.0, 6.0]"
2,toilets,float64,9,0,"[5.0, 6.0, 3.0, 1.0, 7.0]"
3,parking_space,float64,9,0,"[4.0, 6.0, 2.0, 1.0, 5.0]"
4,title,object,7,0,"[Detached Duplex, Semi Detached Duplex, Detach..."
5,town,object,189,0,"[Mabushi, Lekki, Ajah, Epe, Lokogoma District]"
6,state,object,25,0,"[Abuja, Lagos, Edo, Ogun, Oyo]"
7,price,float64,450,0,"[450000000.0, 120000000.0, 40000000.0, 7500000..."


In [76]:
data.to_csv('../data/interim/house_data.csv', index=False)